# SparkSQL and DataFrames 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

## RDDs, DataSets, and DataFrames

RDDs are the original interface for Spark programming.

DataFrames were introduced in 1.3

Datasets were introduced in 1.6, and unified with DataFrames in 2.0

### Advantages of DataFrames:

from https://www.datacamp.com/community/tutorials/apache-spark-python:

> More specifically, the performance improvements are due to two things, which you’ll often come across when you’re reading up DataFrames: custom memory management (project Tungsten), which will make sure that your Spark jobs much faster given CPU constraints, and optimized execution plans (Catalyst optimizer), of which the logical plan of the DataFrame is a part.

## SparkSQL and DataFrames 


pyspark does not have the Dataset API, which is available only if you use Spark from a statically typed language: Scala or Java.

From https://spark.apache.org/docs/2.2.0/sql-programming-guide.html:

> A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. The DataFrame API is available in Scala, Java, Python, and R. In Scala and Java, a DataFrame is represented by a Dataset of Rows. In the Scala API, DataFrame is simply a type alias of Dataset[Row]. While, in Java API, users need to use Dataset&lt;Row> to represent a DataFrame.


### The pyspark.sql module

Important classes of Spark SQL and DataFrames:

* `pyspark.sql.SparkSession` Main entry point for DataFrame and SQL functionality.

* `pyspark.sql.DataFrame` A distributed collection of data grouped into named columns.

* `pyspark.sql.Column` A column expression in a DataFrame.

* `pyspark.sql.Row` A row of data in a DataFrame.

* `pyspark.sql.GroupedData` Aggregation methods, returned by DataFrame.groupBy().

* `pyspark.sql.DataFrameNaFunctions` Methods for handling missing data (null values).

* `pyspark.sql.DataFrameStatFunctions` Methods for statistics functionality.

* `pyspark.sql.functions` List of built-in functions available for DataFrame.

* `pyspark.sql.types` List of data types available.

* `pyspark.sql.Window` For working with window functions.

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

## SparkSession

The traditional way to interact with Spark is the SparkContext. In the notebooks we get that from the pyspark driver.

From 2.0 we can use SparkSession to replace SparkConf, SparkContext and SQLContext

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
from pyspark.sql import SparkSession

session = SparkSession.builder.getOrCreate()

In [3]:
session.sparkContext

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
# Así es como se meten opciones y se cambian
session = SparkSession.builder.config("config.option.key", "value").getOrCreate()

session.sparkContext.getConf().getAll()

[('spark.app.id', 'local-1536995346147'),
 ('config.option.key', 'value'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '10.0.2.15'),
 ('spark.driver.port', '42069'),
 ('spark.app.name', 'PySparkShell')]

#### Passing other options to spark session:
    
    

We can check option values in the resulting session like this:

### Creating DataFrames

SparkSession.createDataFrame: from an RDD, a list or a pandas.DataFrame.

In [ ]:
session.createDataFrame()

### Creating DataFrames

* From lists
* From RDDs
* from Hive tables
* From Spark sources: parquet (default), json, jdbc, orc, libsvm, csv, text


In [6]:
import random

random.seed(42)

ids = range(12)
positions = [random.choice(["teacher", "data scientist", "project manager"]) for _ in ids]
positions

['project manager',
 'teacher',
 'teacher',
 'project manager',
 'data scientist',
 'teacher',
 'teacher',
 'teacher',
 'project manager',
 'teacher',
 'project manager',
 'project manager']

In [7]:
df = session.createDataFrame(zip(ids, positions))

df.collect()

[Row(_1=0, _2='project manager'),
 Row(_1=1, _2='teacher'),
 Row(_1=2, _2='teacher'),
 Row(_1=3, _2='project manager'),
 Row(_1=4, _2='data scientist'),
 Row(_1=5, _2='teacher'),
 Row(_1=6, _2='teacher'),
 Row(_1=7, _2='teacher'),
 Row(_1=8, _2='project manager'),
 Row(_1=9, _2='teacher'),
 Row(_1=10, _2='project manager'),
 Row(_1=11, _2='project manager')]

In [8]:
df.take(5)

[Row(_1=0, _2='project manager'),
 Row(_1=1, _2='teacher'),
 Row(_1=2, _2='teacher'),
 Row(_1=3, _2='project manager'),
 Row(_1=4, _2='data scientist')]

In [9]:
df.show(5)

+---+---------------+
| _1|             _2|
+---+---------------+
|  0|project manager|
|  1|        teacher|
|  2|        teacher|
|  3|project manager|
|  4| data scientist|
+---+---------------+
only showing top 5 rows



In [10]:
from pyspark.sql import Row

help(Row)

Help on class Row in module pyspark.sql.types:

class Row(builtins.tuple)
 |  A row in L{DataFrame}.
 |  The fields in it can be accessed:
 |  
 |  * like attributes (``row.key``)
 |  * like dictionary values (``row[key]``)
 |  
 |  ``key in row`` will search through row keys.
 |  
 |  Row can be used to create a row object by using named arguments,
 |  the fields will be sorted by names. It is not allowed to omit
 |  a named argument to represent the value is None or missing. This should be
 |  explicitly set to None in this case.
 |  
 |  >>> row = Row(name="Alice", age=11)
 |  >>> row
 |  Row(age=11, name='Alice')
 |  >>> row['name'], row['age']
 |  ('Alice', 11)
 |  >>> row.name, row.age
 |  ('Alice', 11)
 |  >>> 'name' in row
 |  True
 |  >>> 'wrong_key' in row
 |  False
 |  
 |  Row also can be used to create another Row like class, then it
 |  could be used to create Row objects, such as
 |  
 |  >>> Person = Row("name", "age")
 |  >>> Person
 |  <Row(name, age)>
 |  >>> 'name' 

Esto es para definir el esquema, es poner el nombre y tipo de las columnas

In [11]:
rows = [Row(id=id, position=position) for id, position in zip(ids, positions)]
rows

[Row(id=0, position='project manager'),
 Row(id=1, position='teacher'),
 Row(id=2, position='teacher'),
 Row(id=3, position='project manager'),
 Row(id=4, position='data scientist'),
 Row(id=5, position='teacher'),
 Row(id=6, position='teacher'),
 Row(id=7, position='teacher'),
 Row(id=8, position='project manager'),
 Row(id=9, position='teacher'),
 Row(id=10, position='project manager'),
 Row(id=11, position='project manager')]

In [12]:
df = session.createDataFrame(rows)
df

DataFrame[id: bigint, position: string]

In [13]:
session.createDataFrame(zip(ids, positions), schema = ["employee_id", "employee_position"])

DataFrame[employee_id: bigint, employee_position: string]

#### From RDDs

In [14]:
rdd = session.sparkContext.textFile("coupon150720.csv")
rdd

coupon150720.csv MapPartitionsRDD[22] at textFile at NativeMethodAccessorImpl.java:0

In [15]:
rdd.take(5)

['79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 '79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 '79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 '79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 '79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

Vemos que cada cupón es una string

Para separarlo vamos a hacer split. Y no hacemos flatmap, hacemos map

In [16]:
split_rdd = rdd.map(lambda line: line.split(","))
split_rdd.take(20)

[['79062005698500',
  '1',
  'MAA',
  'AUH',
  '9W',
  '9W',
  '56.79',
  'USD',
  '1',
  'H',
  'H',
  '0526',
  '150904',
  'OK',
  'IAF0'],
 ['79062005698500',
  '2',
  'AUH',
  'CDG',
  '9W',
  '9W',
  '84.34',
  'USD',
  '1',
  'H',
  'H',
  '6120',
  '150905',
  'OK',
  'IAF0'],
 ['79062005924069',
  '1',
  'CJB',
  'MAA',
  '9W',
  '9W',
  '60.0',
  'USD',
  '1',
  'H',
  'H',
  '2768',
  '150721',
  'OK',
  'IAA0'],
 ['79065668570385',
  '1',
  'DEL',
  'DXB',
  '9W',
  '9W',
  '160.63',
  'USD',
  '2',
  'S',
  'S',
  '0546',
  '150804',
  'OK',
  'INA0'],
 ['79065668737021',
  '1',
  'AUH',
  'IXE',
  '9W',
  '9W',
  '152.46',
  'USD',
  '1',
  'V',
  'V',
  '0501',
  '150803',
  'OK',
  'INA0'],
 ['79062006192650',
  '1',
  'RPR',
  'BOM',
  '9W',
  '9W',
  '68.5',
  'USD',
  '1',
  'K',
  'K',
  '2202',
  '150720',
  'OK',
  'IAE0'],
 ['79062006192650',
  '2',
  'BOM',
  'RPR',
  '9W',
  '9W',
  '68.5',
  'USD',
  '1',
  'H',
  'H',
  '0377',
  '150721',
  'OK',
  'IAE0'],


In [17]:
session.createDataFrame(split_rdd)

DataFrame[_1: string, _2: string, _3: string, _4: string, _5: string, _6: string, _7: string, _8: string, _9: string, _10: string, _11: string, _12: string, _13: string, _14: string, _15: string]

Vemos que no hay nombres ni tipos

### Inferring and specifying schemas

In [18]:
from pyspark.sql import types

types.IntegerType()
types.StructType()

StructType(List())

In [19]:
types.StructField("id", types.IntegerType(), nullable=False) # Nullable igual a False quiere decir que es obligatorio que esté relleno

StructField(id,IntegerType,false)

In [20]:
types.StructField("position", types.StringType())

StructField(position,StringType,true)

In [21]:
schema = types.StructType([types.StructField("id", types.IntegerType(), nullable=False),
                  types.StructField("position", types.StringType())])

In [22]:
df = session.createDataFrame(zip(ids, positions), schema = schema)
df

DataFrame[id: int, position: string]

In [23]:
df.printSchema()

root
 |-- id: integer (nullable = false)
 |-- position: string (nullable = true)



Ahora vemos que el data frame tiene el nombre y el tipo que queremos

#### Fully specifying a schema

We need to create a `StructType` composed of `StructField`s. each of those specifies afiled with name, type and `nullable` properties. 

#### From csv files

We can either read them directly into dataframes or read them as RDDs and transform that into a DataFrame. This second way will be very useful if we have unstructured data like web server logs.

In [24]:
df_from_csv = session.read.csv("coupon150720.csv", inferSchema=True)
df_from_csv.show(2)

+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|  _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W|56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W|84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
only showing top 2 rows



In [25]:
df_from_csv.printSchema()

root
 |-- _c0: long (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)



In [26]:
session.sql("SELECT * FROM csv. `coupon150720.csv`")

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string]

In [27]:
session.sql("SELECT * FROM csv.`coupon150720.csv` WHERE _c2='MAD'").show(5)

+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|  _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|79062005639642|  1|MAD|BRU| 9W| 9W|15.39|USD|  1|  O|   O|6032|150902|  OK|INB0|
|79065668754871|  1|MAD|BRU| SN| 9W|18.56|USD|  1|  W|   W|6032|150921|  OK|INB0|
|79062006141366|  1|MAD|CDG| AF| AF|60.92|USD|  1|  K|   K|1401|150806|  OK|INB0|
|79062006133090|  1|MAD|CDG| AF| AF| 3.38|USD|  1|  X|   X|1101|151027|  OK|IAE0|
|79062006137072|  1|MAD|CDG| AF| AF|26.45|USD|  1|  V|   V|1801|150802|  OK|INA0|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
only showing top 5 rows



#### From other types of data

Apache Parquet is a free and open-source column-oriented data store of the Apache Hadoop ecosystem. It is similar to the other columnar storage file formats available in Hadoop namely RCFile and Optimized RCFile. It is compatible with most of the data processing frameworks in the Hadoop environment.

### Basic operations with DataFrames

In [28]:
df

DataFrame[id: int, position: string]

In [29]:
df.take(5)

[Row(id=0, position='project manager'),
 Row(id=1, position='teacher'),
 Row(id=2, position='teacher'),
 Row(id=3, position='project manager'),
 Row(id=4, position='data scientist')]

In [30]:
df.show(5)

+---+---------------+
| id|       position|
+---+---------------+
|  0|project manager|
|  1|        teacher|
|  2|        teacher|
|  3|project manager|
|  4| data scientist|
+---+---------------+
only showing top 5 rows



In [31]:
df.printSchema()

root
 |-- id: integer (nullable = false)
 |-- position: string (nullable = true)



### Filtering and selecting

Syntax inspired in SQL.

In [32]:
df.select("position").show(5)

+---------------+
|       position|
+---------------+
|project manager|
|        teacher|
|        teacher|
|project manager|
| data scientist|
+---------------+
only showing top 5 rows



If we want to filter, we will need to build an instance of `Column`, using square bracket notation.

In [33]:
df.filter(df["id"] > 5).show()

+---+---------------+
| id|       position|
+---+---------------+
|  6|        teacher|
|  7|        teacher|
|  8|project manager|
|  9|        teacher|
| 10|project manager|
| 11|project manager|
+---+---------------+



`where` is exactly synonimous with `filter`

In [34]:
df.where(df["id"] > 5).show()

+---+---------------+
| id|       position|
+---+---------------+
|  6|        teacher|
|  7|        teacher|
|  8|project manager|
|  9|        teacher|
| 10|project manager|
| 11|project manager|
+---+---------------+



In [35]:
col = df["id"] > 5
col

Column<b'(id > 5)'>

In [36]:
col.collect()

TypeError: 'Column' object is not callable

That's because a comparison between str and int will error out, so spark will not even get the chance to infer to which column we are referring.

A column is quite different to a Pandas Series. It is just a reference to a column, and can only be used to construct sparkSQL expressions (select, where...). It can't be collected or taken as a one-dimensional sequence:

#### Exercise

Extract all employee ids which correspond to data scientist

In [37]:
df.filter(df["position"] == "data scientist").select("id").show()

+---+
| id|
+---+
|  4|
+---+



### Adding columns

Dataframes are immutable, since they are built on top of RDDs, so we can not assign to them. We need to create new DataFrames with the appropriate columns.

In [38]:
newdf = df.withColumn("square", df["id"] ** 2)
newdf.show()

+---+---------------+------+
| id|       position|square|
+---+---------------+------+
|  0|project manager|   0.0|
|  1|        teacher|   1.0|
|  2|        teacher|   4.0|
|  3|project manager|   9.0|
|  4| data scientist|  16.0|
|  5|        teacher|  25.0|
|  6|        teacher|  36.0|
|  7|        teacher|  49.0|
|  8|project manager|  64.0|
|  9|        teacher|  81.0|
| 10|project manager| 100.0|
| 11|project manager| 121.0|
+---+---------------+------+



In [39]:
newdf = df.select("id",
                  "position",
                  (df["id"] ** 2).alias("square"))
newdf.show()

+---+---------------+------+
| id|       position|square|
+---+---------------+------+
|  0|project manager|   0.0|
|  1|        teacher|   1.0|
|  2|        teacher|   4.0|
|  3|project manager|   9.0|
|  4| data scientist|  16.0|
|  5|        teacher|  25.0|
|  6|        teacher|  36.0|
|  7|        teacher|  49.0|
|  8|project manager|  64.0|
|  9|        teacher|  81.0|
| 10|project manager| 100.0|
| 11|project manager| 121.0|
+---+---------------+------+



### User defined functions

There are many useful functions in pyspark.sql.functions. These work on columns, that is, they are vectorial.

We can write User Defined Functions (`udf`s), which allow us to "vectorize" operations: write a standard function to process single elements, then build a udf with that that works on columns in a DataFrame, like a SQL function.

In [40]:
from pyspark.sql import functions

help(functions)

Help on module pyspark.sql.functions in pyspark.sql:

NAME
    pyspark.sql.functions - A collections of builtin functions

FUNCTIONS
    abs(col)
        Computes the absolute value.
        
        .. versionadded:: 1.3
    
    acos(col)
        Computes the cosine inverse of the given value; the returned angle is in the range0.0 through pi.
        
        .. versionadded:: 1.4
    
    add_months(start, months)
        Returns the date that is `months` months after `start`
        
        >>> df = spark.createDataFrame([('2015-04-08',)], ['d'])
        >>> df.select(add_months(df.d, 1).alias('d')).collect()
        [Row(d=datetime.date(2015, 5, 8))]
        
        .. versionadded:: 1.5
    
    approxCountDistinct(col, rsd=None)
        .. note:: Deprecated in 2.1, use approx_count_distinct instead.
        
        .. versionadded:: 1.3
    
    approx_count_distinct(col, rsd=None)
        Returns a new :class:`Column` for approximate distinct count of ``col``.
        
     

In [41]:
import math

newdf.select("id",
             "position",
             math.sqrt(newdf["square"]))

TypeError: must be real number, not Column

In [42]:
newdf.select("id",
             "position",
             functions.sqrt("square"))

DataFrame[id: int, position: string, SQRT(square): double]

This errors out because 

```python
math.log1p
```

is not a udf: it doesn't know how to work with strings or Column objects:

But we can transform it into a udf:

We can do the same with any function we dream up:

In [43]:
da_truth = functions.udf(lambda field: "awesome" if field == "data scientist" else "loser")
da_truth

<function __main__.<lambda>>

In [44]:
df.select("id",
          "position",
          da_truth("position"))

DataFrame[id: int, position: string, <lambda>(position): string]

In [45]:
cooldf = df.select("id",
                   "position",
                   da_truth("position").alias("amen"))

cooldf.show()

+---+---------------+-------+
| id|       position|   amen|
+---+---------------+-------+
|  0|project manager|  loser|
|  1|        teacher|  loser|
|  2|        teacher|  loser|
|  3|project manager|  loser|
|  4| data scientist|awesome|
|  5|        teacher|  loser|
|  6|        teacher|  loser|
|  7|        teacher|  loser|
|  8|project manager|  loser|
|  9|        teacher|  loser|
| 10|project manager|  loser|
| 11|project manager|  loser|
+---+---------------+-------+



If we want the resulting columns to be of a particular type, we need to specify the return type. This is because in Python return types can not be inferred.

In [46]:
doubler = functions.udf(lambda n: n + n)

cooldf.select(doubler("id"))

DataFrame[<lambda>(id): string]

In [47]:
doubler = functions.udf(lambda n: n + n, types.IntegerType())

cooldf.select(doubler("id"))

DataFrame[<lambda>(id): int]

#### Exercise: 

Create a 'salary' field in our df. make it 20000 for teachers, 40000 for product manager and 7e5 for data scientist.



In [48]:
cooldf.show()

+---+---------------+-------+
| id|       position|   amen|
+---+---------------+-------+
|  0|project manager|  loser|
|  1|        teacher|  loser|
|  2|        teacher|  loser|
|  3|project manager|  loser|
|  4| data scientist|awesome|
|  5|        teacher|  loser|
|  6|        teacher|  loser|
|  7|        teacher|  loser|
|  8|project manager|  loser|
|  9|        teacher|  loser|
| 10|project manager|  loser|
| 11|project manager|  loser|
+---+---------------+-------+



In [49]:
def salary(position):
    
    salary_table = {"data scientist" : 700000, "project manager" : 40000, "teacher" : 20000}
        
    return salary_table[position]

salary("teacher")

20000

In [50]:
salary_udf = functions.udf(salary)
salary_udf

<function __main__.salary>

In [51]:
exercise_result = cooldf.select("id", 
                                "position",
                                "amen",
                                salary_udf("position"))

exercise_result.show()

+---+---------------+-------+----------------+
| id|       position|   amen|salary(position)|
+---+---------------+-------+----------------+
|  0|project manager|  loser|           40000|
|  1|        teacher|  loser|           20000|
|  2|        teacher|  loser|           20000|
|  3|project manager|  loser|           40000|
|  4| data scientist|awesome|          700000|
|  5|        teacher|  loser|           20000|
|  6|        teacher|  loser|           20000|
|  7|        teacher|  loser|           20000|
|  8|project manager|  loser|           40000|
|  9|        teacher|  loser|           20000|
| 10|project manager|  loser|           40000|
| 11|project manager|  loser|           40000|
+---+---------------+-------+----------------+



In [52]:
exercise_result.printSchema()

root
 |-- id: integer (nullable = false)
 |-- position: string (nullable = true)
 |-- amen: string (nullable = true)
 |-- salary(position): string (nullable = true)



If we have a column that is not the desired type, we can convert it with `cast`.

In [53]:
exercise_result = cooldf.select("id", 
                                "position",
                                "amen",
                                salary_udf("position").cast(types.IntegerType()).alias("sueldo"))
exercise_result.show()

+---+---------------+-------+------+
| id|       position|   amen|sueldo|
+---+---------------+-------+------+
|  0|project manager|  loser| 40000|
|  1|        teacher|  loser| 20000|
|  2|        teacher|  loser| 20000|
|  3|project manager|  loser| 40000|
|  4| data scientist|awesome|700000|
|  5|        teacher|  loser| 20000|
|  6|        teacher|  loser| 20000|
|  7|        teacher|  loser| 20000|
|  8|project manager|  loser| 40000|
|  9|        teacher|  loser| 20000|
| 10|project manager|  loser| 40000|
| 11|project manager|  loser| 40000|
+---+---------------+-------+------+



In [54]:
exercise_result.printSchema()

root
 |-- id: integer (nullable = false)
 |-- position: string (nullable = true)
 |-- amen: string (nullable = true)
 |-- sueldo: integer (nullable = true)



### Summary statistics

https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html

In [117]:
exercise_result.stat.corr("id", "sueldo")

-0.12091276966734668

### .crosstab()

Crosstab returns the contingency table for two columns, as a DataFrame.

La tabla de contingencia es una tabla donde se ven las relaciones de las variables

In [55]:
loc_udf = functions.udf(lambda: random.choice(["Madrid", "Barcelona"]))

In [56]:
df2 = exercise_result.withColumn("location", loc_udf())
df2

DataFrame[id: int, position: string, amen: string, sueldo: int, location: string]

In [58]:
df2.cache()
df2.show()

+---+---------------+-------+------+---------+
| id|       position|   amen|sueldo| location|
+---+---------------+-------+------+---------+
|  0|project manager|  loser| 40000|   Madrid|
|  1|        teacher|  loser| 20000|   Madrid|
|  2|        teacher|  loser| 20000|Barcelona|
|  3|project manager|  loser| 40000|Barcelona|
|  4| data scientist|awesome|700000|Barcelona|
|  5|        teacher|  loser| 20000|Barcelona|
|  6|        teacher|  loser| 20000|Barcelona|
|  7|        teacher|  loser| 20000|Barcelona|
|  8|project manager|  loser| 40000|Barcelona|
|  9|        teacher|  loser| 20000|   Madrid|
| 10|project manager|  loser| 40000|   Madrid|
| 11|project manager|  loser| 40000|   Madrid|
+---+---------------+-------+------+---------+



In [59]:
df2.crosstab("position", "location").show()

+-----------------+---------+------+
|position_location|Barcelona|Madrid|
+-----------------+---------+------+
|  project manager|        2|     3|
|   data scientist|        1|     0|
|          teacher|        4|     2|
+-----------------+---------+------+



La tabla de contingencias es para ver si dos variables son independientes

### Grouping

Grouping works very similarly to Pandas: executing groupby (or groupBy) on a DataFrame will return an object (a GroupedData) that can then be aggregated to obtain the results.

In [61]:
groups = df2.groupby("location")
groups

GroupedData has several aggregation functions defined:

In [64]:
groups.avg("sueldo").show()

+---------+------------------+
| location|       avg(sueldo)|
+---------+------------------+
|   Madrid|           32000.0|
|Barcelona|122857.14285714286|
+---------+------------------+



We can do several aggregations in a single step, with a number of different syntaxes:

In [65]:
groups.agg({"sueldo" : "mean", "position" : "min"}).show()

+---------+------------------+---------------+
| location|       avg(sueldo)|  min(position)|
+---------+------------------+---------------+
|   Madrid|           32000.0|project manager|
|Barcelona|122857.14285714286| data scientist|
+---------+------------------+---------------+



### Intersections

Veru much like SQL joins. We can specify the columns and the join method (left, right, inner, outer) or we can let Spark infer them.

In [67]:
random.seed(42)

data = list(zip(
[10, 11, 9, 10, 9],
    [5000, 10000, 2000, 0, 1000],
    [random.choice(["Madrid", "Barcelona", "Sevilla"]) for _ in range(5)]))

data

[(10, 5000, 'Sevilla'),
 (11, 10000, 'Madrid'),
 (9, 2000, 'Madrid'),
 (10, 0, 'Sevilla'),
 (9, 1000, 'Barcelona')]

In [69]:
right_df = session.createDataFrame(data, schema = ["id", "bonus", "location"])
right_df.show()

+---+-----+---------+
| id|bonus| location|
+---+-----+---------+
| 10| 5000|  Sevilla|
| 11|10000|   Madrid|
|  9| 2000|   Madrid|
| 10|    0|  Sevilla|
|  9| 1000|Barcelona|
+---+-----+---------+



In [70]:
df2.join(right_df).show()

AnalysisException: 'Detected cartesian product for INNER join between logical plans\nInMemoryRelation [id#51, position#52, amen#305, sueldo#342, location#356], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)\n   +- *Project [id#51, position#52, pythonUDF0#373 AS amen#305, cast(pythonUDF1#374 as int) AS sueldo#342, pythonUDF2#375 AS location#356]\n      +- BatchEvalPython [<lambda>(position#52), salary(position#52), <lambda>()], [id#51, position#52, pythonUDF0#373, pythonUDF1#374, pythonUDF2#375]\n         +- Scan ExistingRDD[id#51,position#52]\nand\nLogicalRDD [id#637L, bonus#638L, location#639]\nJoin condition is missing or trivial.\nUse the CROSS JOIN syntax to allow cartesian products between these relations.;'

Spark refuses to do cross joins by default. To perform them, we can 

a) Allow then explicitly:

```python
session.conf.set("spark.sql.crossJoin.enabled", "true")
```

b) Specify the join criterion

```python
df4.join(new_df, on='id').show()
```

In [71]:
df2.join(right_df, on = "id").show()

+---+---------------+-----+------+--------+-----+---------+
| id|       position| amen|sueldo|location|bonus| location|
+---+---------------+-----+------+--------+-----+---------+
| 10|project manager|loser| 40000|  Madrid| 5000|  Sevilla|
| 11|project manager|loser| 40000|  Madrid|10000|   Madrid|
|  9|        teacher|loser| 20000|  Madrid| 2000|   Madrid|
| 10|project manager|loser| 40000|  Madrid|    0|  Sevilla|
|  9|        teacher|loser| 20000|  Madrid| 1000|Barcelona|
+---+---------------+-----+------+--------+-----+---------+



In [76]:
joined = df2.join(right_df, on = ["id", "location"], how = "left").cache()
joined.show()

+---+---------+---------------+-------+------+-----+
| id| location|       position|   amen|sueldo|bonus|
+---+---------+---------------+-------+------+-----+
|  3|Barcelona|project manager|  loser| 40000| null|
|  9|   Madrid|        teacher|  loser| 20000| 2000|
|  5|Barcelona|        teacher|  loser| 20000| null|
|  0|   Madrid|project manager|  loser| 40000| null|
|  7|Barcelona|        teacher|  loser| 20000| null|
|  6|Barcelona|        teacher|  loser| 20000| null|
| 10|   Madrid|project manager|  loser| 40000| null|
|  2|Barcelona|        teacher|  loser| 20000| null|
| 11|   Madrid|project manager|  loser| 40000|10000|
|  8|Barcelona|project manager|  loser| 40000| null|
|  4|Barcelona| data scientist|awesome|700000| null|
|  1|   Madrid|        teacher|  loser| 20000| null|
+---+---------+---------------+-------+------+-----+



#### Digression

We can monitor our running jobs and storage used at the Spark Web UI. We can get its url with sc.uiWebUrl.

StorageLevels represent how our DataFrame is cached: we can save the results of the computation up to that point, so that if we process several times the same data only the subsequent steps will be recomputed.

In [73]:
sc.uiWebUrl

'http://10.0.2.15:4040'

We can erase it with `unpersist`

In [77]:
joined.unpersist()

DataFrame[id: int, location: string, position: string, amen: string, sueldo: int, bonus: bigint]

#### Exercise

Calculate the [z-score](http://www.statisticshowto.com/probability-and-statistics/z-score/) of each employee's salary for their location

1) Calculate the mean and std of salary for each location

In [91]:
aggregated = joined.groupby("location").agg(functions.avg("sueldo").alias("avg_sueldo"),
                                            functions.stddev("sueldo").alias("std_sueldo"))

aggregated.show()

+---------+------------------+------------------+
| location|        avg_sueldo|        std_sueldo|
+---------+------------------+------------------+
|   Madrid|           32000.0|10954.451150103323|
|Barcelona|122857.14285714286|254670.65566559267|
+---------+------------------+------------------+



2) Annotate each employee with the stats corresponding to their location

In [93]:
joined_2 = joined.join(aggregated, on = "location", how = "left")
joined_2.show()

+---------+---+---------------+-------+------+-----+------------------+------------------+
| location| id|       position|   amen|sueldo|bonus|        avg_sueldo|        std_sueldo|
+---------+---+---------------+-------+------+-----+------------------+------------------+
|   Madrid|  9|        teacher|  loser| 20000| 2000|           32000.0|10954.451150103323|
|   Madrid|  0|project manager|  loser| 40000| null|           32000.0|10954.451150103323|
|   Madrid| 10|project manager|  loser| 40000| null|           32000.0|10954.451150103323|
|   Madrid| 11|project manager|  loser| 40000|10000|           32000.0|10954.451150103323|
|   Madrid|  1|        teacher|  loser| 20000| null|           32000.0|10954.451150103323|
|Barcelona|  3|project manager|  loser| 40000| null|122857.14285714286|254670.65566559267|
|Barcelona|  5|        teacher|  loser| 20000| null|122857.14285714286|254670.65566559267|
|Barcelona|  7|        teacher|  loser| 20000| null|122857.14285714286|254670.65566559267|

3) Calculate the z-score

In [98]:
result = joined_2.withColumn("z_score", (joined_2["sueldo"] - joined_2["avg_sueldo"]) / joined_2["std_sueldo"])
result.show()

+---------+---+---------------+-------+------+-----+------------------+------------------+--------------------+
| location| id|       position|   amen|sueldo|bonus|        avg_sueldo|        std_sueldo|             z_score|
+---------+---+---------------+-------+------+-----+------------------+------------------+--------------------+
|   Madrid|  9|        teacher|  loser| 20000| 2000|           32000.0|10954.451150103323| -1.0954451150103321|
|   Madrid|  0|project manager|  loser| 40000| null|           32000.0|10954.451150103323|  0.7302967433402214|
|   Madrid| 10|project manager|  loser| 40000| null|           32000.0|10954.451150103323|  0.7302967433402214|
|   Madrid| 11|project manager|  loser| 40000|10000|           32000.0|10954.451150103323|  0.7302967433402214|
|   Madrid|  1|        teacher|  loser| 20000| null|           32000.0|10954.451150103323| -1.0954451150103321|
|Barcelona|  3|project manager|  loser| 40000| null|122857.14285714286|254670.65566559267|-0.32535017684

In [103]:
result.sort(-result["z_score"]).show()

+---------+---+---------------+-------+------+-----+------------------+------------------+--------------------+
| location| id|       position|   amen|sueldo|bonus|        avg_sueldo|        std_sueldo|             z_score|
+---------+---+---------------+-------+------+-----+------------------+------------------+--------------------+
|Barcelona|  4| data scientist|awesome|700000| null|122857.14285714286|254670.65566559267|   2.266232266275318|
|   Madrid| 11|project manager|  loser| 40000|10000|           32000.0|10954.451150103323|  0.7302967433402214|
|   Madrid| 10|project manager|  loser| 40000| null|           32000.0|10954.451150103323|  0.7302967433402214|
|   Madrid|  0|project manager|  loser| 40000| null|           32000.0|10954.451150103323|  0.7302967433402214|
|Barcelona|  3|project manager|  loser| 40000| null|122857.14285714286|254670.65566559267|-0.32535017684150597|
|Barcelona|  8|project manager|  loser| 40000| null|122857.14285714286|254670.65566559267|-0.32535017684

Note that we can build more complex boolean conditions for joining, as well as joining on columns that do not have the same name:

### Handling null values

In [104]:
joined.dropna().show()

+---+--------+---------------+-----+------+-----+
| id|location|       position| amen|sueldo|bonus|
+---+--------+---------------+-----+------+-----+
|  9|  Madrid|        teacher|loser| 20000| 2000|
| 11|  Madrid|project manager|loser| 40000|10000|
+---+--------+---------------+-----+------+-----+



## SQL querying

We need to register our DataFrame as a table in the SQL context in order to be able to query against it.

In [106]:
result.registerTempTable("result_table")

session.sql("SELECT * FROM result_table WHERE sueldo > 30000").show()

+---------+---+---------------+-------+------+-----+------------------+------------------+--------------------+
| location| id|       position|   amen|sueldo|bonus|        avg_sueldo|        std_sueldo|             z_score|
+---------+---+---------------+-------+------+-----+------------------+------------------+--------------------+
|   Madrid|  0|project manager|  loser| 40000| null|           32000.0|10954.451150103323|  0.7302967433402214|
|   Madrid| 10|project manager|  loser| 40000| null|           32000.0|10954.451150103323|  0.7302967433402214|
|   Madrid| 11|project manager|  loser| 40000|10000|           32000.0|10954.451150103323|  0.7302967433402214|
|Barcelona|  3|project manager|  loser| 40000| null|122857.14285714286|254670.65566559267|-0.32535017684150597|
|Barcelona|  8|project manager|  loser| 40000| null|122857.14285714286|254670.65566559267|-0.32535017684150597|
|Barcelona|  4| data scientist|awesome|700000| null|122857.14285714286|254670.65566559267|   2.266232266

Once registered, we can perform queries as complex as we want.

In [107]:
session.sql("SELECT id, position, sqrt(sueldo) FROM result_table WHERE sueldo > 30000").show()

+---+---------------+----------------------------+
| id|       position|SQRT(CAST(sueldo AS DOUBLE))|
+---+---------------+----------------------------+
|  0|project manager|                       200.0|
| 10|project manager|                       200.0|
| 11|project manager|                       200.0|
|  3|project manager|                       200.0|
|  8|project manager|                       200.0|
|  4| data scientist|           836.6600265340755|
+---+---------------+----------------------------+



In [109]:
session.udf.register("salary_registered", salary_udf())

session.sql("SELECT id, position, salary_registered(position) FROM result_table WHERE sueldo > 30000").show()

TypeError: Not a function or callable (__call__ is not defined): <class 'pyspark.sql.column.Column'>

#### Exercise:

replicate the previous exercise, but with SparkSQL instead of dataframe methods.

## Interoperation with Pandas

Easy peasy. We can convert a spark DataFrame into a Pandas one, which will `collect` it, and viceversa, which will distribute it.

In [110]:
df = joined.toPandas()

In [111]:
df

,id,location,position,amen,sueldo,bonus
0,3,Barcelona,project manager,loser,40000,NaN
1,9,Madrid,teacher,loser,20000,2000.0
2,5,Barcelona,teacher,loser,20000,NaN
3,0,Madrid,project manager,loser,40000,NaN
4,7,Barcelona,teacher,loser,20000,NaN
5,6,Barcelona,teacher,loser,20000,NaN
6,10,Madrid,project manager,loser,40000,NaN
7,2,Barcelona,teacher,loser,20000,NaN
8,11,Madrid,project manager,loser,40000,10000.0
9,8,Barcelona,project manager,loser,40000,NaN


In [112]:
type(df)

pandas.core.frame.DataFrame

In [113]:
session.createDataFrame(df)

DataFrame[id: bigint, location: string, position: string, amen: string, sueldo: bigint, bonus: double]

## Writing out


In [114]:
joined.write.csv("joined.csv")
# Esto me ha generado una carpeta con diversos archivos

#### Exercise

Repeat the exercise from the previous notebook, but this time with DataFrames.

Get stats for all tickets with destination MAD from `coupons150720.csv`.

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

1) Extract the fields you need (c0,c1,c2,c3,c4 and c6) into a dataframe with proper names and types

Remember, you want to calculate:

Total ticket amounts per origin

Top 10 airlines by average amount

2) Total ticket amounts per origin

3) Top 10 Airlines by average amount



## Further Reading

https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

https://www.datacamp.com/community/tutorials/apache-spark-python

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

https://ogirardot.wordpress.com/2015/05/29/rdds-are-the-new-bytecode-of-apache-spark/

https://stackoverflow.com/questions/36822224/what-are-the-pros-and-cons-of-parquet-format-compared-to-other-formats

https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf